In [10]:
import mxnet as mx

from mxnet import nd, autograd, gluon
from mxnet.gluon.nn import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Sequential, HybridSequential

import numpy as np

import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout

from datetime import datetime

ctx = mx.cpu()

## Buggy

Very slow to run, probably due to incorrect data loading
GPU is not really working much...

In [11]:
batch_size = 128
num_inputs = 784
num_outputs = 10

def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)

train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

In [22]:
def add_conv_layer(x, num_filter, act_type="tanh"):
    
    x.add(Conv2D(channels=num_filter, kernel_size=5, activation=act_type))
    x.add(MaxPool2D(pool_size=2, strides=2))

    return


def add_full_layers(x, architecture, act_type="tanh", p=None):
    
    for num_cell in architecture:
        x.add(Dense(num_cell, activation=act_type))
        if p is not None:
            x.add(Dropout(p=p))

    return


def generate_lenet(dropout=None, act_type="tanh", full_geom=[500], hybridize=True):
    
    net = HybridSequential(prefix="lenet_")

    with net.name_scope():
        add_conv_layer(net, num_filter=20, act_type=act_type)
        add_conv_layer(net, num_filter=50, act_type=act_type)
        net.add(Flatten())
        add_full_layers(net, full_geom, act_type=act_type)
        net.add(Dense(num_outputs))
    
    if hybridize:
        net.hybridize()
    
    return net

In [14]:
lenet = generate_lenet()

lenet

HybridSequential(
  (0): Conv2D(20, kernel_size=(5, 5), stride=(1, 1))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (2): Conv2D(50, kernel_size=(5, 5), stride=(1, 1))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (4): Flatten
  (5): Dense(500, Activation(tanh))
  (6): Dense(10, linear)
)

In [27]:
x = mx.sym.var("data")

print(lenet(x).tojson())

{
  "nodes": [
    {
      "op": "null", 
      "name": "data", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "lenet_conv0_weight", 
      "attr": {
        "__dtype__": "0", 
        "__lr_mult__": "1.0", 
        "__shape__": "(20, 0, 5, 5)", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "lenet_conv0_bias", 
      "attr": {
        "__dtype__": "0", 
        "__init__": "zeros", 
        "__lr_mult__": "1.0", 
        "__shape__": "(20,)", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "Convolution", 
      "name": "lenet_conv0_fwd", 
      "attr": {
        "dilate": "(1, 1)", 
        "kernel": "(5, 5)", 
        "layout": "NCHW", 
        "no_bias": "False", 
        "num_filter": "20", 
        "num_group": "1", 
        "pad": "(0, 0)", 
        "stride": "(1, 1)"
      }, 
      "inputs": [[0, 0, 0], [1, 0, 0], [2, 0, 0]]
    }, 
    {
      "op": "Activati

In [15]:
%%time

lenet.collect_params().initialize(mx.init.Xavier(magnitude=2), ctx=ctx)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

trainer = gluon.Trainer(lenet.collect_params(), 'sgd', {'learning_rate': 0.1})

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.06 ms


In [16]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [17]:
def run_epochs(epochs=2):

    smoothing_constant = 0.01

    start = datetime.now()

    for e in range(epochs):
        for i, (data, label) in enumerate(train_data):
            beg = datetime.now()
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            with autograd.record():
                output = lenet(data)
                loss = softmax_cross_entropy(output, label)
            loss.backward()
            trainer.step(data.shape[0])

            curr_loss = nd.mean(loss).asscalar()
            moving_loss = (curr_loss if ((i==0) and (e==0))
                           else (1 - smoothing_constant) * moving_loss + smoothing_constant * curr_loss)

        test_accuracy = evaluate_accuracy(test_data, lenet)
        train_accuracy = evaluate_accuracy(train_data, lenet)
        print("@ %s: Epoch %d. Loss: %.4f, train acc %.4f, test_acc %.4f" % (
            datetime.now() - beg, e, moving_loss, train_accuracy, test_accuracy))

mx.profiler.profiler_set_config(mode='all', filename='/tmp/prf.json')
mx.profiler.profiler_set_state('run')

run_epochs()

mx.profiler.profiler_set_state("stop")

In [18]:
run_epochs()

KeyboardInterrupt: 